<a href="https://colab.research.google.com/github/ProfAI/nlp00/blob/master/8%20-%20Chatbot/dante_RNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!wget https://www.liberliber.it/mediateca/libri/a/alighieri/la_divina_commedia/txt/la_divin.zip

--2019-04-20 16:52:32--  https://www.liberliber.it/mediateca/libri/a/alighieri/la_divina_commedia/txt/la_divin.zip
Resolving www.liberliber.it (www.liberliber.it)... 93.186.244.67
Connecting to www.liberliber.it (www.liberliber.it)|93.186.244.67|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 232691 (227K) [application/zip]
Saving to: ‘la_divin.zip.1’

la_divin.zip.1      100%[===================>] 227.24K   466KB/s    in 0.5s    

2019-04-20 16:52:33 (466 KB/s) - ‘la_divin.zip.1’ saved [232691/232691]



In [22]:
!unzip la_divin.zip

Archive:  la_divin.zip
replace la_divin.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [1]:
with open("la_divin.txt", encoding="latin-1") as divine_file:
  divine_txt = divine_file.read()
  
print(divine_txt[:100])

Dante Alighieri
La Divina Commedia

Questo e-book è stato realizzato anche grazie al sostegno di:
E-


In [2]:
start = divine_txt.find("Nel mezzo del cammin di nostra vita")
end = divine_txt.find("l'amor che move il sole e l'altre stelle.")

divine_txt = divine_txt[start:end]

print(divine_txt[:100])

Nel mezzo del cammin di nostra vita
mi ritrovai per una selva oscura,
ché la diritta via era smarrit


In [0]:
import re

divine_txt = re.sub("[\(\[].*?[\)\]]", "", divine_txt)
divine_txt = re.sub("[A-Z][A-Z]+","",divine_txt)

divine_txt = divine_txt.replace("di Dante Alighieri","")

divine_txt = re.sub(r'[^\w\s]','',divine_txt)
divine_txt = divine_txt.replace("\n"," ")
divine_txt = divine_txt.lower()

In [4]:
!python -m spacy download it_core_news_sm


    Linking successful
    /usr/local/lib/python3.6/dist-packages/it_core_news_sm -->
    /usr/local/lib/python3.6/dist-packages/spacy/data/it_core_news_sm

    You can now load the model via spacy.load('it_core_news_sm')



In [5]:
import spacy

nlp = spacy.load("it_core_news_sm")

def preprocess(text):

  tokens = nlp(text)
  tokens_filtered = [token.text for token in tokens]
  return tokens_filtered

tokens = preprocess(divine_txt)
tokens[:10]

['nel',
 'mezzo',
 'del',
 'cammin',
 'di',
 'nostra',
 'vita',
 'mi',
 'ritrovai',
 'per']

In [6]:
maxlen = 10

divine_sents = []

for i in range(maxlen, len(tokens)):
  divine_sents.append(tokens[i-maxlen:i])
  
print(divine_sents[0])
print(divine_sents[1])

['nel', 'mezzo', 'del', 'cammin', 'di', 'nostra', 'vita', 'mi', 'ritrovai', 'per']
['mezzo', 'del', 'cammin', 'di', 'nostra', 'vita', 'mi', 'ritrovai', 'per', 'una']


In [7]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(divine_sents)
divine_sents = tokenizer.texts_to_sequences(divine_sents)

divine_sents[0]

Using TensorFlow backend.


[38, 224, 24, 603, 4, 186, 153, 15, 13574, 7]

In [0]:
import numpy as np
divine_sents = np.array(divine_sents)

X = divine_sents[:,:-1]
Y = divine_sents[:,-1]

In [9]:
from keras.utils import to_categorical

vocab_size = len(tokenizer.word_counts)

Y = to_categorical(Y, num_classes=vocab_size+1)
Y.shape

(97393, 13575)

In [10]:
from keras import Model, Sequential
from keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(vocab_size+1, maxlen-1, input_length=maxlen-1))
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50, activation="relu"))
model.add(Dense(vocab_size+1, activation="softmax"))

Instructions for updating:
Colocations handled automatically by placer.


In [0]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

In [12]:
model.fit(X, Y, batch_size=128, epochs=100)

Instructions for updating:
Use tf.cast instead.
Epoch 1/100
97393/97393 [==============================] - 41s 420us/step - loss: 7.4553 - acc: 0.0392
Epoch 2/100
97393/97393 [==============================] - 37s 384us/step - loss: 7.0885 - acc: 0.0407
Epoch 3/100
97393/97393 [==============================] - 37s 376us/step - loss: 6.9740 - acc: 0.0458
Epoch 4/100
97393/97393 [==============================] - 37s 385us/step - loss: 6.8586 - acc: 0.0538
Epoch 5/100
97393/97393 [==============================] - 37s 379us/step - loss: 6.7773 - acc: 0.0569
Epoch 6/100
97393/97393 [==============================] - 37s 383us/step - loss: 6.7014 - acc: 0.0587
Epoch 7/100
97393/97393 [==============================] - 36s 374us/step - loss: 6.6215 - acc: 0.0606
Epoch 8/100
97393/97393 [==============================] - 37s 384us/step - loss: 6.5462 - acc: 0.0625
Epoch 9/100
97393/97393 [==============================] - 37s 379us/step - loss: 6.4741 - acc: 0.0641
Epoch 10/100
97393/97393 

In [0]:
from keras.preprocessing.sequence import pad_sequences

def dante_says(text, output_len=25):
  
    output = ""
  
    for i in range(output_len):
      tokens = np.array(tokenizer.texts_to_sequences([text]))
      tokens = pad_sequences(tokens, maxlen=maxlen-1, truncating="pre")
      
      pred_word = model.predict_classes([tokens])[0]
      pred_word = tokenizer.index_word[pred_word]
      
      text+=" "+pred_word
      output+=pred_word+" "
    
    return output

In [20]:
text = "oggi è una bella giornata"

answer = dante_says(text)

print("Giuseppe: "+text)
print("Dante: "+answer)

Giuseppe: oggi è una bella giornata
Dante: solaio e odono e domanda è fui ancor io che la vegg io mi disse che l mondo che l corpo dolorando stringonsi e quei 


# PARCHEGGIO